# Sentiment Classification & Embedding II

* Embedding Layer
* Sequence Model

# 01. What data we use?

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
url = 'https://raw.githubusercontent.com/bab2min/corpus/master/sentiment/steam.txt'
data= pd.read_table(url, names=['label', 'reviews'])

In [ ]:
data.head()

In [ ]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(data, test_size=0.2, random_state=2021)
train_data.reset_index(drop=True, inplace=True)
test_data.reset_index(drop=True, inplace=True)

In [ ]:
train_data.shape, test_data.shape

In [ ]:
test_data.head()

# Q1. Tokenizing & Text to Sequences

* 상위 40000개 단어만 사용한다.

In [ ]:
## 위의 한국어 전처리를 했다면 사용하지 않는다.
x_train = train_data['reviews'].astype('str').tolist()
x_test = test_data['reviews'].astype('str').tolist()

In [ ]:
y_train = train_data['label'].values
y_test = test_data['label'].values

In [ ]:
### Tokenizer here




In [ ]:
### Tokenizer here




In [ ]:
# Text --> Sequence



# Padding Sequence

* 문장의 최대 길이는 40으로 한다.

In [ ]:
from tensorflow.keras.preprocessing.sequence import p

In [ ]:
#### Pad Sequences here




In [ ]:
x_train = np.array(x_train)
x_test = np.array(x_test)

# 0으로 가득찬 데이터에 대한 추가 전처리

In [ ]:
not0train_idx = x_train.sum(1) > 0
not0test_idx = x_test.sum(1) > 0

In [ ]:
train_data = train_data.loc[not0train_idx].reset_index(drop=True)
test_data = test_data.loc[not0test_idx].reset_index(drop=True)

In [ ]:
x_train = x_train[not0train_idx]
y_train = y_train[not0train_idx]

x_test = x_test[not0test_idx]
y_test = y_test[not0test_idx]

# 모델링

**모델 구조**
1. 임베딩 레이어 : 임베딩차원은 128
2. Conv1D 블록 : 필터수 64개, 윈도우 사이즈 5
3. Bidirectional layer :
    * 정방향 : LSTM, 히든스테이트 32 
    * 역방향 : LSTM, 히든스테이트 32
4. Bidirectional layer :
    * 정방향 : GRU, 히든스테이트 32
    * 역방향 : RNN, 히든스테이트 16
5. Conv1D 블록 : 필터수 32개, 윈도우 사이즈 5
6. MaxPool1D 블록 : 필터사이즈2
7. 플래튼
8. FC Layer : 노드 1024개
9. 시그모이드 레이어

In [ ]:
#####################
## your codes here ##
#####################





In [ ]:
#####################
## your codes here ##
#####################






# EarlyStopping을 이용한 학습.

1. 20%는 벨리데이션 셋.
2. 4epochs전과 비교하여 early stopping할 것.

In [ ]:
#####################
## your codes here ##
#####################




In [ ]:
#####################
## your codes here ##
#####################






In [ ]:
review_idx = 128

temp = test_data.loc[review_idx]
docu = temp['reviews']
label = 'positive' if temp['label'] ==1 else 'Negative'

print(f"문서 번호 {review_idx}")
print(label, " : ", docu)
y_pred = model.predict(x_test[review_idx:review_idx+1])
label_pred = 'positive' if y_pred[0,0] >=0.5 else 'Negative'
print(f"모델의 예측 : {label_pred},   prob = {y_pred[0,0]*100:.2f}%")